<a href="https://colab.research.google.com/github/jkurian49/Brushstroke-Classifier/blob/main/Brushstroke_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/jkurian49/Brushstroke-Classifier.git
%cd Brushstroke-Classifier/

Cloning into 'Brushstroke-Classifier'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 113 (delta 4), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (113/113), 113.49 KiB | 8.73 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/Brushstroke-Classifier


# Data Loading and Preprocessing 

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.preprocessing import image

data = pd.read_csv('dummy-data.csv')
data.replace('', np.nan, inplace=True)
data.dropna(inplace=True)

# CHANGE IMAGE PATH TO WORK IN GITHUB
image_path = "dummy-images/"

# Preprocess training images
TARGET_SIZE = (300,300,3) # make (128,128) for b&w images
train_image = []

for i in tqdm(range(data.shape[0])): 
    img = image.load_img(image_path + data['ID'][i] + '.jpg',target_size=TARGET_SIZE)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

print(X.shape)
# plt.imshow(X[2])
# specify variable to predict
data = data['Brush Material']
y = np.array(data)
print(y.shape)

100%|██████████| 99/99 [00:00<00:00, 1119.77it/s]

(99, 300, 300, 3)
(99,)


In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=420, test_size=0.2)

# Training Model

In [28]:
# PARAMETERS
NUM_CLASSES = 4
NUM_EPOCHS = 100
BATCH_SIZE = 16

In [26]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow import keras
# load model without classifier layers
model = VGG16(include_top=False, input_shape=(300, 300, 3))
# mark loaded layers as not trainable
for layer in model.layers:
	layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(NUM_CLASSES, activation='softmax')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# summarize
# model.summary()

model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [30]:
print("Fit model on training data")
history = model.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(X_valid, y_valid),
)

Fit model on training data
Epoch 1/100
5/5 [==============================] - 0s 106ms/step - loss: 0.0013 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.8321 - val_sparse_categorical_accuracy: 0.3500
Epoch 2/100
5/5 [==============================] - 0s 97ms/step - loss: 0.0013 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.8743 - val_sparse_categorical_accuracy: 0.3500
Epoch 3/100
5/5 [==============================] - 0s 97ms/step - loss: 0.0012 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.7871 - val_sparse_categorical_accuracy: 0.3000
Epoch 4/100
4/5 [=======================>......] - ETA: 0s - loss: 0.0013 - sparse_categorical_accuracy: 1.0000

KeyboardInterrupt: ignored